# Accidents de voitures et météo

Par Anthony Gerbier et Mathias Pisch

# Introduction

Bla bla bla

<a id="sommaire"></a>
## Sommaire
- [Installation](#installation)
- [Préparation des données](#préparation)
  - [Adresses](#url)
  - [Accidents de voitures dans le Maryland](#car_accidents)
    - [Chargement des données](#download)
    - [Nettoyage des données](#cleaning)
    - [Création d'un sample](#sample)
  - [Données météo](#weather)
    - [Appels API](#weather_api)
    - [Variable Jour/Nuit](#day_or_night)
  - [Grille spatio-temporelle](#grid)
    - [Découpage](#grid_creation)
    - [Appels API](#grid_api)
    - [Valeurs moyennes](#grid_average)
- [Analyse descriptive](#descriptive)
  - [Affichage sur une carte](#map)
  - [Corrélation entre les variables et les accidents de voitures](#visual_correlation)
- [Modélisation](#modélisation)
  - [Centrage et réduction des variables](#centrage) --> à mettre dans préparation des données ?
  - [Matrice de corrélation](#matrix)
  - [Régression linéaire](#linear)
  - [Régression logit](#logit)
  - [Comparaison entre les grilles](#grid_comparison)
- [Conclusion et perspectives](#conclusion)

<a id="préparation"></a>
# I. Préparation des données
<a id="url"></a>
## 1. Adresses

In [ ]:
car_accidents_url = "..."

## 1. Downloading the xls data from the public url

### [mettre le code de Anthony]

## 2. Adding the weather to the sample dataset

We are using the free API of Open Weather, which provides weather data from 1940 until now. 
The API call has different parameters: some are required (latitude, longitude, start_date, end_date), and other are optional (elevation -to improve accuracy, apikey -required only for commercial use, hourly -a list of hourly weather variables which should be returned, daily -a list of daily weather variable aggregations which should be returned).

Limitations: Only for non-commercial use and less than 10.000 daily API calls.

## Hourly variables choosen (description below)
- weather_code (WMO code)
- temperature_2m (°C)
- precipitation (mm)
- snowfall (cm)
- relative_humidity_2m (%)

## Daily variables choosen (description below)
- sunrise
- sunset

## Linear regression (Mettre ceci plutôt dans le readme, ou dans la partie correspondante du NoteBook weather)
We are planning to regress the number of accident on :
- weather_code (WMO code) : a standarzed code between 1 and 99, depending on the weather -the higher it is, the harsher the weather is
- precipitation (mm) : Total precipitation (rain, showers, snow) sum of the preceding hour
- snowfall (cm) : Snowfall amount of the preceding hour in centimeters. For the water equivalent in millimeter, divide by 7. E.g. 7 cm snow = 10 mm precipitation water equivalent
- wind_speed_10m (km/h) : Wind speed at 10 or 100 meters above ground. Wind speed on 10 meters is the standard level.
- night (0 or 1 if it is the night) : this is a variable that we creates using the two daily variables sunrise and sunset

## Focus on the WMO
The conditions corresponding to the WMO code are described here: https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM.

Weather icons illustration for each code are provided here: https://gist.github.com/stellasphere/9490c195ed2b53c707087c8c2db4ec0c

## DOCS for the Open Meteo
https://open-meteo.com/en/docs/historical-weather-api

### Reading the csv file with the data about car accident without weather

In [42]:
import pandas as pd
import os 

# Defining the columns to read
#usecols = ["...

# Read data with subset of columns

# if the file "completed_dataset_sample.csv" exists, we load it instead of the initial dataset, to avoid redoing already completed work
if os.path.exists("completed_dataset_sample.csv"):
    car_accidents_data = pd.read_csv("completed_dataset_sample.csv")
else:
    car_accidents_data = pd.read_csv("bigbase_sample_clean.csv") # usecols=usecols) to select specific columns

# Preview first 5 rows
car_accidents_data.head()

,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Report Number,Local Case Number,Agency Name,ACRS Report Type,...,weather_code,precipitation,snowfall,wind_speed,wind_direction,soil_temperature,soil_moisture,sunrise,sunset,day_or_night_code
0,0,0,0,0,0,0,MCP3130004M,220015468,Montgomery County Police,Property Damage Crash,...,51.0,0.3,0.0,10.829959,285.422211,11.233001,0.459,05:35:00,18:42:00,0
1,1,1,1,1,1,1,MCP29520027,16031710,Montgomery County Police,Injury Crash,...,63.0,3.2,0.0,4.320000,270.000000,21.872499,0.518,04:43:00,19:37:00,0
2,2,2,2,2,2,2,MCP1048001D,16011658,MONTGOMERY,Property Damage Crash,...,2.0,0.0,0.0,6.120000,241.927612,8.493999,0.439,06:28:00,18:09:00,0
3,3,3,3,3,3,3,MCP2667004Q,180023612,Montgomery County Police,Injury Crash,...,3.0,0.0,0.0,3.319036,139.398788,15.491500,0.407,04:55:00,19:13:00,0
4,4,4,4,4,4,4,MCP3030003D,200018469,Montgomery County Police,Property Damage Crash,...,61.0,1.5,0.0,12.599998,323.130005,12.625000,0.506,05:04:00,19:06:00,0


# Create squares 

In [43]:
def create_squares(df,lat,long,nb_sq_lat,nb_sq_long):
    lat_min = 38.934343
    lat_max = 39.353502
    lon_min = -77.52768
    lon_max = -76.888505
    grid_dict = {}
    
    # Compute the step sizes
    lat_step = (lat_max - lat_min) / nb_sq_lat
    lon_step = (lon_max - lon_min) / nb_sq_long
    
    for i in range(nb_sq_lat):
        for j in range(nb_sq_long):
            square_lat_min = lat_min + i * lat_step
            square_lat_max = lat_min + (i + 1) * lat_step
            square_lon_min = lon_min + j * lon_step
            square_lon_max = lon_min + (j + 1) * lon_step
            
            grid_dict[(i, j)] = (square_lat_min, square_lat_max, square_lon_min, square_lon_max)

    return grid_dict

create_squares(car_accidents_data,"Latitude","Longitude",3,3)

{(0, 0): (38.934343, 39.07406266666666, -77.52768, -77.31462166666667),
 (0, 1): (38.934343,
  39.07406266666666,
  -77.31462166666667,
  -77.10156333333333),
 (0, 2): (38.934343, 39.07406266666666, -77.10156333333333, -76.888505),
 (1, 0): (39.07406266666666,
  39.213782333333334,
  -77.52768,
  -77.31462166666667),
 (1, 1): (39.07406266666666,
  39.213782333333334,
  -77.31462166666667,
  -77.10156333333333),
 (1, 2): (39.07406266666666,
  39.213782333333334,
  -77.10156333333333,
  -76.888505),
 (2, 0): (39.213782333333334, 39.353502, -77.52768, -77.31462166666667),
 (2, 1): (39.213782333333334,
  39.353502,
  -77.31462166666667,
  -77.10156333333333),
 (2, 2): (39.213782333333334, 39.353502, -77.10156333333333, -76.888505)}

### Let's add for each line of the dataset in which square we are 

for each cell, we have to find in which square we are 

In [44]:
def find_square_from_grid(lat, lon, grid_dict):
    """
    Return the grid cell (i, j) containing the point (lat, lon).
    """
    for (i, j), (lat_min, lat_max, lon_min, lon_max) in grid_dict.items():
        if lat_min <= lat < lat_max and lon_min <= lon < lon_max:
            return (i, j)
    return None




ajouter au data set le numéro du carré

In [45]:
grid = create_squares(
    car_accidents_data,
    "Latitude",
    "Longitude",
    3,
    3
)


car_accidents_data['grid_cell'] = car_accidents_data.apply(
    lambda row: find_square_from_grid(
        row['Latitude'],
        row['Longitude'],
        grid
    ),
    axis=1
)
print(grid.keys())
car_accidents_data[['Latitude', 'Longitude', 'grid_cell']].head()



dict_keys([(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)])


,Latitude,Longitude,grid_cell
0,39.084912,-77.076068,"(1, 2)"
1,39.013298,-77.045822,"(0, 2)"
2,39.026372,-77.204397,"(0, 1)"
3,39.176243,-77.118350,"(1, 1)"
4,38.992689,-77.161965,"(0, 1)"


In [47]:
print(len(grid))
#print(grid[0])
print(grid)
print(grid.keys())

9
{(0, 0): (38.934343, 39.07406266666666, -77.52768, -77.31462166666667), (0, 1): (38.934343, 39.07406266666666, -77.31462166666667, -77.10156333333333), (0, 2): (38.934343, 39.07406266666666, -77.10156333333333, -76.888505), (1, 0): (39.07406266666666, 39.213782333333334, -77.52768, -77.31462166666667), (1, 1): (39.07406266666666, 39.213782333333334, -77.31462166666667, -77.10156333333333), (1, 2): (39.07406266666666, 39.213782333333334, -77.10156333333333, -76.888505), (2, 0): (39.213782333333334, 39.353502, -77.52768, -77.31462166666667), (2, 1): (39.213782333333334, 39.353502, -77.31462166666667, -77.10156333333333), (2, 2): (39.213782333333334, 39.353502, -77.10156333333333, -76.888505)}
dict_keys([(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)])


In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd

lat_min = 38.934343
lat_max = 39.353502
lon_min = -77.52768
lon_max = -76.888505
# Create geometry column from Latitude / Longitude
geometry = [
    Point(lon, lat)
    for lon, lat in zip(
        car_accidents_data['Longitude'],
        car_accidents_data['Latitude']
    )
]

gdf = gpd.GeoDataFrame(
    car_accidents_data,
    geometry=geometry,
    crs="EPSG:4326"
)

polygons = []
cells = []

for (i, j), (lat_min, lat_max, lon_min, lon_max) in grid.items():
    poly = Polygon([
        (lon_min, lat_min),
        (lon_max, lat_min),
        (lon_max, lat_max),
        (lon_min, lat_max)
    ])
    polygons.append(poly)
    cells.append(f"({i}, {j})")

grid_gdf = gpd.GeoDataFrame(
    {'grid_cell': cells},
    geometry=polygons,
    crs="EPSG:4326"
)

gdf['grid_cell_str'] = gdf['grid_cell'].astype(str)

fig, ax = plt.subplots(figsize=(10, 10))

# Plot grid boundaries
grid_gdf.boundary.plot(
    ax=ax,
    color='black',
    linewidth=1
)

# Plot accident points colored by grid cell
gdf.plot(
    ax=ax,
    column='grid_cell_str',
    cmap='tab10',
    markersize=8,
    legend=True,
    alpha=0.7
)

ax.set_title("Accident locations colored by grid cell", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

plt.show()


ModuleNotFoundError: No module named 'geopandas'

## Computing the average temperature for each square of the grid for each month

In [103]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timezone, timedelta

url = "https://archive-api.open-meteo.com/v1/archive"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


#creer un tableau ca

def month_weather_api(latitude, longitude, start_date, end_date):
# on fournit la lat et long du centre du carré, et la date, afin d'obtenir.

# The order of variables in hourly or daily is important (to assign them correctly below)

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "UTC-5",
        "daily": [
        "temperature_2m_min",
        "temperature_2m_max",
        "temperature_2m_mean",
        "weather_code",
        "apparent_temperature_max",
        "apparent_temperature_min",
        "precipitation_sum",
        "rain_sum",
        "snowfall_sum",
        "precipitation_hours",
        "sunrise",
        "sunset",
        "sunshine_duration",
        "daylight_duration",
        "wind_speed_10m_max",
        "wind_gusts_10m_max",
        "wind_direction_10m_dominant",
        "shortwave_radiation_sum",
        "et0_fao_evapotranspiration"
    ]
    }

    responses = openmeteo.weather_api(url, params=params)
    #print(responses)

    response = responses[0]
    # Process daily data. The order of variables needs to be the same as requested.
    
    daily = response.Daily()
    #print(daily)

    daily_min_temp= daily.Variables(0).ValuesAsNumpy()
    daily_max_temp = daily.Variables(1).ValuesAsNumpy()
    daily_mean_temp = daily.Variables(2).ValuesAsNumpy()
    weather_code = daily.Variables(3).ValuesAsNumpy()
    apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
    apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
    precipitation_sum = daily.Variables(6).ValuesAsNumpy()
    rain_sum = daily.Variables(7).ValuesAsNumpy()
    snowfall_sum = daily.Variables(8).ValuesAsNumpy()
    precipitation_hours = daily.Variables(9).ValuesAsNumpy()
    sunrise = daily.Variables(10).ValuesInt64AsNumpy()
    sunset = daily.Variables(11).ValuesInt64AsNumpy()

    ## ou si on préfère avoir les données dans le format plus explicite: (on préfère garder le format précédent, en nombre de secondes écoulés depuis 1er janvier 1970)
    
    #raw_sunrise = daily.Variables(10).ValuesInt64AsNumpy()
    #raw_sunset = daily.Variables(11).ValuesInt64AsNumpy()

    #list_sunrise = raw_sunrise.tolist()
    #list_sunset = raw_sunset.tolist()

    #utc_minus_5 = timezone(timedelta(hours=-5))

    #sunrise = [datetime.fromtimestamp(e,tz=utc_minus_5).isoformat() for e in list_sunrise]
    
    #sunset = [datetime.fromtimestamp(e, tz=utc_minus_5).isoformat() for e in list_sunset]
    
    sunshine_duration = daily.Variables(12).ValuesAsNumpy()
    daylight_duration = daily.Variables(13).ValuesAsNumpy()
    wind_speed_10m_max = daily.Variables(14).ValuesAsNumpy()
    wind_gusts_10m_max = daily.Variables(15).ValuesAsNumpy()
    wind_direction_10m_dominant = daily.Variables(16).ValuesAsNumpy()
    shortwave_radiation_sum = daily.Variables(17).ValuesAsNumpy()
    et0_fao_evapotranspiration = daily.Variables(18).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["temperature_2m_min"] = daily_min_temp
    daily_data["temperature_2m_max"] = daily_max_temp
    daily_data["temperature_2m_mean"] = daily_mean_temp
    daily_data["weather_code"] = weather_code
    daily_data["apparent_temperature_max"] = apparent_temperature_max
    daily_data["apparent_temperature_min"] = apparent_temperature_min
    daily_data["precipitation_sum"] = precipitation_sum
    daily_data["rain_sum"] = rain_sum
    daily_data["snowfall_sum"] = snowfall_sum
    daily_data["precipitation_hours"] = precipitation_hours
    daily_data["sunrise"] = sunrise
    daily_data["sunset"] = sunset
    daily_data["sunshine_duration"] = sunshine_duration
    daily_data["daylight_duration"] = daylight_duration
    daily_data["wind_speed_10m_max"] = wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = shortwave_radiation_sum
    daily_data["et0_fao_evapotranspiration"] = et0_fao_evapotranspiration

    return {"daily": daily_data}

#weather_api(52.52, 13.41, "2025-11-09", "2025-11-09")

#month_weather_api(39.2903848, -76.6121893, "2025-01-01", "2025-01-02")
#add_weather()


## pour le rapport:
# convertion du format iso8601 vers un autre pour le sunset et sunrise
# mise sous forme d'un dictonnaire de dictionnaires (un pour les hourly, un pour les daily (ici besoin que du daily, mais dans la fonction plus bas d'ajout de la météo au dataset on utilise les 2 types de données: hourly et daily))
# pas de vérification qu'on a pas de NaN dans l'appel car tous les problèmes ont été réglés à la main donc il n'y en a plus, mais ça pourrait être bien.



### Using the month_weather_api function to fill the array

In [50]:
cells = list(grid.keys())

nb_cells_col = (cells[-1])[1]+1 #pour calculer l'index des cellules

months_nb = 12
years = list(range(2014, 2026))
years_nb = len(years)

nb_variables = 19 #the number of variables that we stock for the monthly dataset

### Ne pas relancer ceci (les appels API pour récupérer les données brutes)

In [ ]:
import numpy as np
import calendar # to get the number of days in a month for the call of the month_weather_api
import time #for the .sleep()

cells = list(grid.keys())

nb_cells_col = (cells[-1])[1]+1 #pour calculer l'index des cellules

months_nb = 12
years = list(range(2014, 2026))
years_nb = len(years)

nb_variables = 19 #the number of variables that we stock for the monthly dataset

raw_data = np.zeros(
    (len(cells), months_nb, years_nb, nb_variables),
    dtype=list
)

for c in grid :
    cell_index = c[0]*nb_cells_col + c[1]
    # on calcule le centre de la cellule (lat, long)
    cell = grid[c]
    l1, l2, L1, L2 = cell [0], cell[1], cell[2], cell[3]
    lcenter, Lcenter = (l2 + l1)/2, (L2 + L1)/2
    print("Starting cell:", c)
    print("Central latitude:", lcenter)
    print("Central longitude:", Lcenter)

    for m in range(months_nb) :
        m_arg = str(m + 1)
        if m+1 < 10:
            m_arg = "0" + m_arg

        for y in years:

            y_index = 2025 - y #we obtain the index number, starting at 0 for 2025
            #print(y)
            #y_arg = str(y)[2] + str(y)[3]
            #print(y_arg)

            last_day = calendar.monthrange(y, m+1)[1] #+1 because month must be between 1 and 12 for the monthrange
            #print(last_day)

            start_date = str(y) + "-" + m_arg + "-" + "01"

            end_date = str(y) + "-" + m_arg + "-"  + str(last_day)
            
            if y == 2025 and m == 11 :
                end_date = "2025-12-19" #the historcal API only allow past data
        
            print(start_date)
            print(end_date)

            # etape 1: recuperer données brutes via la fonction
            raw_variables = month_weather_api(lcenter, Lcenter, start_date, end_date)
            
            daily_data = raw_variables["daily"]

            # etape 2: faire 3 list à partir des array avec le .tolist(), pour min, max, mean

            min_temp_list = daily_data['temperature_2m_min'].tolist()
            max_temp_list = daily_data['temperature_2m_max'].tolist()
            mean_temp_list = daily_data['temperature_2m_mean'].tolist()
                    
            weather_code_list = daily_data['weather_code'].tolist()

            apparent_temperature_max_list = daily_data['apparent_temperature_max'].tolist()
            apparent_temperature_min_list = daily_data['apparent_temperature_min'].tolist()

            precipitation_sum_list = daily_data['precipitation_sum'].tolist()
            rain_sum_list = daily_data['rain_sum'].tolist()
            snowfall_sum_list = daily_data['snowfall_sum'].tolist()

            precipitation_hours_list = daily_data['precipitation_hours'].tolist()

            sunrise_list = daily_data['sunrise'].tolist()
            sunset_list = daily_data['sunset'].tolist()
            
            sunshine_duration_list = daily_data['sunshine_duration'].tolist()
            daylight_duration_list = daily_data['daylight_duration'].tolist()

            wind_speed_10m_max_list = daily_data['wind_speed_10m_max'].tolist()
            wind_gusts_10m_max_list = daily_data['wind_gusts_10m_max'].tolist()
            wind_direction_10m_dominant_list = daily_data['wind_direction_10m_dominant'].tolist()

            shortwave_radiation_sum_list = daily_data['shortwave_radiation_sum'].tolist()
            et0_fao_evapotranspiration_list = daily_data['et0_fao_evapotranspiration'].tolist()

            raw_data[cell_index][m][y_index][0] = min_temp_list
            raw_data[cell_index][m][y_index][1] = max_temp_list
            raw_data[cell_index][m][y_index][2] = mean_temp_list

            raw_data[cell_index][m][y_index][3] = weather_code_list

            raw_data[cell_index][m][y_index][4] = apparent_temperature_max_list
            raw_data[cell_index][m][y_index][5] = apparent_temperature_min_list

            raw_data[cell_index][m][y_index][6] = precipitation_sum_list
            raw_data[cell_index][m][y_index][7] = rain_sum_list
            raw_data[cell_index][m][y_index][8] = snowfall_sum_list

            raw_data[cell_index][m][y_index][9] = precipitation_hours_list

            raw_data[cell_index][m][y_index][10] = sunrise_list
            raw_data[cell_index][m][y_index][11] = sunset_list

            raw_data[cell_index][m][y_index][12] = sunshine_duration_list
            raw_data[cell_index][m][y_index][13] = daylight_duration_list

            raw_data[cell_index][m][y_index][14] = wind_speed_10m_max_list
            raw_data[cell_index][m][y_index][15] = wind_gusts_10m_max_list
            raw_data[cell_index][m][y_index][16] = wind_direction_10m_dominant_list

            raw_data[cell_index][m][y_index][17] = shortwave_radiation_sum_list
            raw_data[cell_index][m][y_index][18] = et0_fao_evapotranspiration_list

            np.savez_compressed("raw_data_weather.npz",  allow_pickle=True, raw_data=raw_data)
            #print(raw_data)

    time.sleep(60)
    print("=========================")

# tableau de dimension 4

# avec un appel API (i.e. de la fonction month_weather_api), on remplit une ligne du tableau (une ligne annee)

# etape 2: faire 3 list à partir des array avec le .tolist(), pour min, max, mean

#data = np.load("raw_data.npz", allow_pickle=True)
#raw_data = data["raw_data"]
#print(raw_data)



Starting cell: (0, 0)
Central latitude: 39.00420283333333
Central longitude: -77.42115083333334
2014-01-01
2014-01-31
2015-01-01
2015-01-31
2016-01-01
2016-01-31
2017-01-01
2017-01-31
2018-01-01
2018-01-31
2019-01-01
2019-01-31
2020-01-01
2020-01-31
2021-01-01
2021-01-31
2022-01-01
2022-01-31
2023-01-01
2023-01-31
2024-01-01
2024-01-31
2025-01-01
2025-01-31
2014-02-01
2014-02-28
2015-02-01
2015-02-28
2016-02-01
2016-02-29
2017-02-01
2017-02-28
2018-02-01
2018-02-28
2019-02-01
2019-02-28
2020-02-01
2020-02-29
2021-02-01
2021-02-28
2022-02-01
2022-02-28
2023-02-01
2023-02-28
2024-02-01
2024-02-29
2025-02-01
2025-02-28
2014-03-01
2014-03-31
2015-03-01
2015-03-31
2016-03-01
2016-03-31
2017-03-01
2017-03-31
2018-03-01
2018-03-31
2019-03-01
2019-03-31
2020-03-01
2020-03-31
2021-03-01
2021-03-31
2022-03-01
2022-03-31
2023-03-01
2023-03-31
2024-03-01
2024-03-31
2025-03-01
2025-03-31
2014-04-01
2014-04-30
2015-04-01
2015-04-30
2016-04-01
2016-04-30
2017-04-01
2017-04-30
2018-04-01
2018-04-30
20

### Correction du type de sunrise et sunset (conversion numpy array --> list pour uniformisation)

In [96]:
from datetime import timezone, timedelta

data = np.load("full_raw_data_weather.npz", allow_pickle=True)
raw_data = data["raw_data"]
utc_minus_5 = timezone(timedelta(hours=-5))

for c in grid :
    cell_index = c[0]*nb_cells_col + c[1]
    # on calcule le centre de la cellule (lat, long)
    cell = grid[c]

    for m in range(months_nb) :

        # pour chacune des variables, on va faire la moyenne des valeurs sur tous les jours du mois, et sur toutes les années
        for y in years:
            y_index = 2025 - y

            list_sunrise = raw_data[cell_index][m][y_index][10].tolist()
            list_sunset = raw_data[cell_index][m][y_index][11].tolist()

            raw_data[cell_index][m][y_index][10] = list_sunrise # ou si conversion souhaitée: [datetime.fromtimestamp(e,tz=utc_minus_5).isoformat() for e in list_sunrise]
            raw_data[cell_index][m][y_index][11] = list_sunset # ou si conversion souhaitée:[datetime.fromtimestamp(e,tz=utc_minus_5).isoformat() for e in list_sunset]
            

np.savez_compressed("full_corrected_raw_data_weather.npz",  allow_pickle=True, raw_data=raw_data)

In [97]:
### Vérification que la correction a bien fonctionnée

data = np.load("full_corrected_raw_data_weather.npz", allow_pickle=True)
raw_data = data["raw_data"]

print(type(raw_data[1][2][5][11]))

<class 'list'>


### Calcul des moyennes à partir des données brutes contenues dans full_raw_data_weather.npz

In [ ]:
data = np.load("full_corrected_raw_data_weather.npz", allow_pickle=True)
raw_data = data["raw_data"]

# creation du tableau permettant de sstocker les valeurs moyennes

# calculs des valeurs moyennes à partir des données brutes
### pour les variables qui nous intéressent:
## 1. Moyenne sur les jours de chaque mois
## 2. Moyenne sur les années de chaque mois
## on obtient les valeurs moyennes pour chaque mois, pour chaque variables, et ce pour chaque cellule de la grille

raw_data_average = np.zeros(
    (len(cells), months_nb, nb_variables),
    dtype=float
)

# calcul des valeurs

for c in grid :
    cell_index = c[0]*nb_cells_col + c[1]
    # on calcule le centre de la cellule (lat, long)
    cell = grid[c]

    for m in range(months_nb):
        m_arg = str(m + 1)
        if m+1 < 10:
            m_arg = "0" + m_arg

        # pour chacune des variables, on va faire la moyenne des valeurs sur tous les jours du mois, et sur toutes les années
        for i in range (0, 19):
            v_acccross_years = []

            for y in years:
                y_index = 2025 - y
                values = raw_data[cell_index][m][y_index][i]

                #if i == 10 or i == 11:
                #    mean = sum(values) / len(values)                            
                #else :
                
                mean = np.mean(raw_data[cell_index][m][y_index][i])

                v_acccross_years.append(mean)
                
            v_mean = np.mean(v_acccross_years)

            raw_data_average[cell_index][m][i] = v_mean
            
np.savez_compressed("full_months_average.npz",  allow_pickle=True, raw_data=raw_data)

## Documentation : Comment utiliser les valeurs moyennes par mois ?

In [ ]:
# pour récupérer ce dataset, il suffit de faire :
data = np.load("full_months_average.npz", allow_pickle=True)
raw_data = data["raw_data"]

## et ensuite pour accéder à la valeur moyenne de la variable i du mois m de la case c:
raw_data_average[cell_index][m][i] = v_mean

# où m = 0 pour janvier, 11 pour décembre
# où cell_index est calculé avec : cell_index = c[0]*nb_cells_col + c[1] pour c in grid.keys()
# et où :
# i = 0  -> min_temp
# i = 1  -> max_temp
# i = 2  -> mean_temp
# i = 3  -> weather_code
# i = 4  -> apparent_temperature_max
# i = 5  -> apparent_temperature_min
# i = 6  -> precipitation_sum
# i = 7  -> rain_sum
# i = 8  -> snowfall_sum
# i = 9  -> precipitation_hours
# i = 10 -> sunrise
# i = 11 -> sunset
# i = 12 -> sunshine_duration
# i = 13 -> daylight_duration
# i = 14 -> wind_speed_10m_max
# i = 15 -> wind_gusts_10m_max
# i = 16 -> wind_direction_10m_dominant
# i = 17 -> shortwave_radiation_sum
# i = 18 -> et0_fao_evapotranspiration

##### Affichage des valeurs moyennes

In [95]:
for c in grid :
    cell_index = c[0]*nb_cells_col + c[1]
    # on calcule le centre de la cellule (lat, long)
    cell = grid[c]
    for m in range(months_nb):
        for i in range (0, 19):
            print(raw_data_average[cell_index][m][i])

-3.223999974908688
5.319951621124582
0.908056529486672
31.776881720430108
1.3310835733208604
-7.77318242672951
2.5803763881245616
1.8819892675445604
0.49395161019938616
3.4274193548387095
1563431106.9247313
1563466369.9623654
21096.771797590358
35262.4481686828
18.401449966174297
38.256773207777286
236.19903559838568
7.758360215412672
1.0896585007668824
-1.6823435735717502
8.315203505131603
3.0923971933766143
33.254002463054185
4.6444337820674
-6.012600337318795
2.9226908406452234
2.2916769158654833
0.4474928057033795
3.8022372742200328
1565989239.332307
1566027966.161741
23593.97752689493
38725.86223292859
18.437769592395558
39.3475850318453
225.1658940699765
10.55044338501048
1.5337182681316321
1.5553010808884775
12.878956687406344
7.033347798441286
29.16935483870968
9.5844933935391
-2.4650932050520376
2.806451665838399
2.3459677836827693
0.32422043258945143
3.3951612903225805
1568546321.096774
1568589449.376344
27975.882091153053
43127.2410219254
19.205016693761273
40.59967634754796

# Ajout de la météo au dataset des accidents

### Code using the API with a function (timezone "UTC-5")

In [14]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timezone, timedelta

url = "https://archive-api.open-meteo.com/v1/archive"

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

def weather_api(latitude, longitude, start_date, end_date):
# Function to get weather data from OpenMeteo API
# Return a json with hourly and daily data

# The order of variables in hourly or daily is important (to assign them correctly below)

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "UTC-5",
        "hourly": ["temperature_2m", "apparent_temperature", "relative_humidity_2m", "weather_code", "precipitation", "snowfall", "wind_speed_10m", "wind_direction_10m", "soil_temperature_0_to_7cm", "soil_moisture_0_to_7cm", "shortwave_radiation", "direct_radiation", "direct_normal_irradiance", "diffuse_radiation", "global_tilted_irradiance", "sunshine_duration"],
        "daily": ["sunrise", "sunset"]
    }

    responses = openmeteo.weather_api(url, params=params)
    #print(responses)

    response = responses[0]

    #print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
    #print(f"Elevation: {response.Elevation()} m asl")
    #print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    temperature = hourly.Variables(0).ValuesAsNumpy()
    apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
    humidity = hourly.Variables(2).ValuesAsNumpy()
    weather_code = hourly.Variables(3).ValuesAsNumpy()
    precipitation = hourly.Variables(4).ValuesAsNumpy()
    snowfall = hourly.Variables(5).ValuesAsNumpy()
    wind_speed = hourly.Variables(6).ValuesAsNumpy()
    wind_direction = hourly.Variables(7).ValuesAsNumpy()
    soil_temperature = hourly.Variables(8).ValuesAsNumpy()
    soil_moisture = hourly.Variables(9).ValuesAsNumpy()
    shortwave_radiation = hourly.Variables(10).ValuesAsNumpy()
    direct_radiation = hourly.Variables(11).ValuesAsNumpy()
    direct_normal_irradiance = hourly.Variables(12).ValuesAsNumpy
    diffuse_radiation = hourly.Variables(13).ValuesAsNumpy()
    global_tilted_irradiance = hourly.Variables(14).ValuesAsNumpy
    sunshine_duration = hourly.Variables(15).ValuesAsNumpy()

    
    daily = response.Daily()

    daily_sunrise = daily.Variables(0).ValuesInt64AsNumpy()
    daily_sunset = daily.Variables(1).ValuesInt64AsNumpy()

    utc_minus_5 = timezone(timedelta(hours=-5))

    daily_sunrise= datetime.fromtimestamp(daily_sunrise[0],tz=utc_minus_5).isoformat()
    daily_sunset= datetime.fromtimestamp(daily_sunset[0], tz=utc_minus_5).isoformat()
    
    
    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["temperature"] = temperature
    hourly_data["apparent_temperature"] = apparent_temperature
    hourly_data["humidity"] = humidity
    hourly_data["weather_code"] = weather_code
    hourly_data["precipitation"] = precipitation
    hourly_data["snowfall"] = snowfall
    hourly_data["wind_speed"] = wind_speed
    hourly_data["wind_direction"] = wind_direction
    hourly_data["soil_temperature"] = soil_temperature
    hourly_data["soil_moisture"] = soil_moisture
    """
    hourly_data["shortwave_radiation"] = shortwave_radiation
    hourly_data["direct_radiation"] = direct_radiation
    hourly_data["direct_normal_irradiance"] = direct_normal_irradiance
    hourly_data["diffuse_radiation"] = diffuse_radiation
    hourly_data["global_tilted_irradiance"] = global_tilted_irradiance
    hourly_data["sunshine_duration"] = sunshine_duration
    """
    
    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["sunrise"] = daily_sunrise
    daily_data["sunset"] = daily_sunset
    

    return {"hourly": hourly_data, "daily": daily_data}

#weather_api(52.52, 13.41, "2025-11-09", "2025-11-09")

weather_api(39.2903848, -76.6121893, "2025-01-01", "2025-01-20")
#add_weather()

{'hourly': {'date': DatetimeIndex(['2025-01-01 05:00:00+00:00', '2025-01-01 06:00:00+00:00',
                 '2025-01-01 07:00:00+00:00', '2025-01-01 08:00:00+00:00',
                 '2025-01-01 09:00:00+00:00', '2025-01-01 10:00:00+00:00',
                 '2025-01-01 11:00:00+00:00', '2025-01-01 12:00:00+00:00',
                 '2025-01-01 13:00:00+00:00', '2025-01-01 14:00:00+00:00',
                 ...
                 '2025-01-20 19:00:00+00:00', '2025-01-20 20:00:00+00:00',
                 '2025-01-20 21:00:00+00:00', '2025-01-20 22:00:00+00:00',
                 '2025-01-20 23:00:00+00:00', '2025-01-21 00:00:00+00:00',
                 '2025-01-21 01:00:00+00:00', '2025-01-21 02:00:00+00:00',
                 '2025-01-21 03:00:00+00:00', '2025-01-21 04:00:00+00:00'],
                dtype='datetime64[ns, UTC]', length=480, freq='h'),
  'temperature': array([ 7.85850000e+00,  8.35850048e+00,  8.45849991e+00,  7.95849991e+00,
          7.55849981e+00,  7.15849972e+00,  6.8084

### Function that add the weather data to the data set

In [ ]:
import time
import os

# if the file "completed_dataset_sample.csv" exists, we load it instead of the initial dataset, to avoid redoing already completed work
if os.path.exists("completed_dataset_sample.csv"):
    car_accidents_data = pd.read_csv("completed_dataset_sample.csv") # usecols=usecols) to select specific columns
else:
    car_accidents_data = pd.read_csv("bigbase_sample_clean.csv") # usecols=usecols) to select specific columns

def add_weather():
    n = len(car_accidents_data)

    # adding weather columns to car_accidents_data

    weather_columns_hourly = [
        "temperature",
        "apparent_temperature",
        "humidity",
        "weather_code",
        "precipitation",
        "snowfall",
        "wind_speed",
        "wind_direction",
        "soil_temperature",
        "soil_moisture"
    ]

    weather_columns_daily = [
        "sunrise",
        "sunset",
        "day_or_night_code"]
    
    weather_columns = weather_columns_hourly + weather_columns_daily

    new_data = {col: [None] * n for col in weather_columns}
    for col, values in new_data.items():
        if col not in car_accidents_data.columns: # to avoid overwriting existing columns with data coming form previous runs
            car_accidents_data[col] = values
    
    #show the full updated dataframe
    #print("Updated car_accidents_data with new weather columns:")
    #print(car_accidents_data.head())
    
    j, k = 0, 0

    start_time = time.perf_counter()
    for i in range(0,n):
        j+=1
        print(f"Processing row {i}, meaning row {j} of the {k} series, out of {n}")
        latitude = car_accidents_data.iloc[i]['Latitude']
        longitude = car_accidents_data.iloc[i]['Longitude']
        date_time = car_accidents_data.iloc[i]['Crash Date/Time']

        date = date_time.split(" ")[0]
        accident_time = date_time.split(" ")[1] #useless
        

        hour = date_time.split(" ")[1].split(":")[0]
        time_of_day = date_time.split(" ")[2]
        time_minutes = date_time.split(" ")[1].split(":")[1]

        time_index = int(hour) # index starts at 0 for midnight, so 1am = 1, 2am = 2, ..., 12pm = 12, 1pm = 13, ..., 11pm = 23

        if time_of_day == "PM" and hour != "12":
            time_index += 12
        elif time_of_day == "AM" and hour == "12":
            time_index = 0

        if time_minutes < "30":
            time_index += 0
        elif int(hour) < 11 :
            time_index += 1

        #print(f"Fetching weather data for row {i} (ID: {car_accidents_data.index[i]}) at coordinates: {latitude}, {longitude}")
        #print(f"Crash Date/Time: {date_time}")
        print(f"Parsed Date: {date}, Time: {accident_time}, Hour: {hour}, Time of Day: {time_of_day}, Time Minutes: {time_minutes}")
        #print(f"Computed Time Index for hourly data: {time_index}")

        date_conversion = pd.to_datetime(date)
        date_for_api = date_conversion.strftime("%Y-%m-%d")
        #print(f"Formatted Date for API: {date_for_api}")

        weather_data = weather_api(latitude, longitude, date_for_api, date_for_api)

        # Extract hourly weather data for the specific hour of the crash
        hourly_data = weather_data["hourly"]
        daily_data = weather_data["daily"]

        #print(hourly_data)
        #print('Daily data:')
        #print(daily_data)

        for col in weather_columns_hourly:
            if col in hourly_data:
                print("Time index:", time_index)
                car_accidents_data.at[i, col] = hourly_data[col][time_index]
                #print(f"Assigned {col} value: {hourly_data[col][time_index]} to row {i} (ID: {car_accidents_data.index[i]})")
            else:
                print(f"Warning: {col} not found in hourly data for row {i} (ID: {car_accidents_data.index[i]})")
            
        col = "day_or_night_code"
        if "sunrise" in daily_data and "sunset" in daily_data:
            sunrise_time = daily_data["sunrise"]
            #print(f"Sunrise time: {sunrise_time}")
            sunrise_time_list = sunrise_time.split("T")[1].split(":")[0:2]
            sunrise_time_h, sunrise_time_m = sunrise_time_list[0], sunrise_time_list[1]
            #print(f"Sunrise time (hour) and minute): {sunrise_time_h} {sunrise_time_m}")

            sunrise_time_hm = sunrise_time_h + ":" + sunrise_time_m + ":00"

            sunset_time = daily_data["sunset"]
            sunset_time_list = sunset_time.split("T")[1].split(":")[0:2]
            sunset_time_h, sunset_time_m = sunset_time_list[0], sunset_time_list[1]
            #print(f"Sunset time (hour and minute): {sunset_time_h} {sunset_time_m}")

            sunset_time_hm = sunset_time_h + ":" + sunset_time_m + ":00"

            if time_of_day == "PM" and hour != "12":
                accident_hour_24h = int(hour) + 12
            elif time_of_day == "AM" and hour == "12":
                accident_hour_24h = 00
            else :
                accident_hour_24h = int(hour)

            if accident_hour_24h < 10:
                accident_hour_24h = "0" + str(accident_hour_24h)
            accident_time_24h = str(accident_hour_24h) + ":" + time_minutes + ":00"

            if sunrise_time_hm <= accident_time_24h <= sunset_time_hm:
                #print(f"Crash time {time} is during the day (between {sunrise_time_hm} and {sunset_time_hm})")
                car_accidents_data.at[i, col] = "0"
            else:
                #print(f"Crash time {time} is during the night (outside {sunrise_time_hm} and {sunset_time_hm})")
                car_accidents_data.at[i, col] = "1"
            
            #print(f"Assigned {col} value: {car_accidents_data.at[i, col]} to row {i} (ID: {car_accidents_data.index[i]})")

            # we add the sunrise and sunset times to the dataframe, because it could be useful later to identify crashes at times close to sunrise/sunset
            car_accidents_data.at[i, "sunrise"] = sunrise_time_hm
            car_accidents_data.at[i, "sunset"] = sunset_time_hm
        else:
            print(f"Warning: sunrise or sunset not found in daily data for row {i} (ID: {car_accidents_data.index[i]})")
        
        if j > 300 :
            # Update the CSV file after each batch of 550 requests
            #car_accidents_data.to_csv("completed_dataset_sample.csv")

            end_time = time.perf_counter()
            elapsed = end_time - start_time
            print(f"Elapsed time for 550 requests: {elapsed} seconds")

            time.sleep(120) # to respect rate limit of 600 requests per minute (with a security margin of 4 seconds)
            j = 0
            k += 1
            start_time = time.perf_counter()

    # Saving the dataframe as a csv file

    # Export the file to the current working directory
    #car_accidents_data.to_csv("completed_dataset_sample.csv")

    return

add_weather()

In [ ]:
car_accidents_data.head()

### Correction of the day_or_night_code column to take PM into account

In [ ]:
if os.path.exists("completed_dataset_sample.csv"):
    car_accidents_data = pd.read_csv("completed_dataset_sample.csv") # usecols=usecols) to select specific columns
else:
    print("The file does not exist.")

n = len(car_accidents_data)
for i in range(0,n):

    date_time = car_accidents_data.iloc[i]['Crash Date/Time']

    accident_time = date_time.split(" ")[1] #useless
    hour = date_time.split(" ")[1].split(":")[0]
    time_of_day = date_time.split(" ")[2]
    time_minutes = date_time.split(" ")[1].split(":")[1]

    if time_of_day == "PM" and hour != "12":
        accident_hour_24h = int(hour) + 12
    elif time_of_day == "AM" and hour == "12":
        accident_hour_24h = 00
    else :
        accident_hour_24h = int(hour)

    if accident_hour_24h < 10:
        accident_hour_24h = "0" + str(accident_hour_24h)
    accident_time_24h = str(accident_hour_24h) + ":" + time_minutes + ":00"

    sunrise_time = car_accidents_data.iloc[i]['sunrise']
    sunset_time = car_accidents_data.iloc[i]['sunset']

    if sunrise_time <= accident_time_24h <= sunset_time:
        car_accidents_data.at[i, "day_or_night_code"] = "0"
        print(f"Daytime crash at row {i}: accident time {accident_time_24h}, sunrise {sunrise_time}, sunset {sunset_time}")
    else:
        car_accidents_data.at[i, "day_or_night_code"] = "1"
        print(f"Nighttime crash at row {i}: accident time {accident_time_24h}, sunrise {sunrise_time}, sunset {sunset_time}")
    
   #print(f"Processed row {i} out of {n}")

car_accidents_data.head()
car_accidents_data.to_csv("completed_dataset_sample.csv")

# Drafts part

In [ ]:
raw_variables = month_weather_api(lcenter, Lcenter, start_date, end_date)
            
daily_data = raw_variables["daily"]

sunrise = daily_data['sunrise']
sunset= daily_data['sunset']

print(type(sunrise))
print(sunrise)

### Code to use the API manually (not required in the final version of the project)

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-11-09",
	"end_date": "2025-11-11",
	"hourly": ["temperature_2m", "relative_humidity_2m", "weather_code", "precipitation", "snowfall" ],
    "daily": ["sunrise", "sunset"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
weather_code = hourly.Variables(2).ValuesAsNumpy()
precipitation = hourly.Variables(3).ValuesAsNumpy()
snowfall = hourly.Variables(4).ValuesAsNumpy()

daily = response.Daily()
daily_sunrise = daily.Variables(0).ValuesAsNumpy()
daily_sunset = daily.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["weather_code"] = weather_code
hourly_data["precipitation"] = precipitation
hourly_data["snowfall"] = snowfall

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)


# Découpage en carrés